# Genetic Algorithm for Timetabling Problem

## 0 - Configurations


## 0.1 - Importing Libraries

In [1]:
from dataclasses import dataclass, Field
from typing import List, Dict, Set, Tuple, Any
import time
import random
import json

## 0.2 - Environmental Variables

In [2]:
MOCK_DATA_FOLDER_PATH = "../../../data/mock/"
PROFESSORS_FILE_PATH = MOCK_DATA_FOLDER_PATH + "teachers.json"
SUBJECTS_FILE_PATH = MOCK_DATA_FOLDER_PATH + "subjects.json"
CLASSROOMS_FILE_PATH = MOCK_DATA_FOLDER_PATH + "classrooms.json"
CLASSGROUPS_FILE_PATH = MOCK_DATA_FOLDER_PATH + "classgroup.json"
DAYS_FILE_PATH = MOCK_DATA_FOLDER_PATH + "days.json"

## 0.3 - Dataclasses

In [3]:
@dataclass
class ClassGroup:
    id: int
    name: str
    students: int

    def __init__(self, class_group_id: int, class_group_data: dict) -> None:
        self.id = class_group_id
        self.name = class_group_data["name"]
        self.students = class_group_data["studentsAmount"]

In [5]:
@dataclass
class Subject:
    id: int
    name: str
    : List[str]
    course_load: int

    def __init__(self, subject_name: str, subject_data: dict) -> None:
        self.id = subject_data["id"]
        self.name = subject_name
        self.professors = subject_data["professors"]
        self.course_load = subject_data["course_load"]

In [4]:
@dataclass
class Professor:
    id: int
    name: str
    subjects_taughtble_id: List[int]
    time_available: dict

    def __init__(self, professor_name: str, professor_data: dict) -> None:
        self.id = professor_data["id"]
        self.name = professor_name
        self.time_available = professor_data["time_available"]

    @property
    def availability(self) -> List[Set[str]]:
        availability = []
        for day, times in self.time_available.items():
            for time in times:
                availability.append((day, time))
        return availability

In [6]:
@dataclass
class Classroom:
    id: int
    name: str
    time_available: Dict[str, List[int]]
    capacity: int
    
    def __init__(self, classroom_name: str, classroom_data: dict) -> None:
        self.id = classroom_data["id"]
        self.name = classroom_name
        self.time_available = classroom_data["time_available"]
        self.capacity = classroom_data["capacity"]

    @property
    def availability(self) -> List[Set[str]]:
        availability = []
        for day, times in self.time_available.items():
            for time in times:
                availability.append((day, time))
        return availability

In [7]:
@dataclass
class Gene:
    classgroup_id: int
    classgroup_students: int
    subject_id: int
    professor_id: int
    professor_day: str
    professor_time: int
    classroom_capacity: int
    
    def __init__(self, classgroup_id: int, classgroup_students: int, subject_id: int, professor_id: int, professor_day: str, professor_time: int, classroom_capacity: int) -> None:
        self.classgroup_id = classgroup_id
        self.classgroup_students = classgroup_students
        self.subject_id = subject_id
        self.professor_id = professor_id
        self.professor_day = professor_day
        self.professor_time = professor_time
        self.classroom_capacity = classroom_capacity

@dataclass
class Chromosome:
    genes: List[Gene]

    def __init__(self) -> None:
        self.genes = []

    def add_gene(self, gene: Gene) -> None:
        self.genes.append(gene)

@dataclass
class Population:
    chromosomes: List[Tuple[Chromosome, float]]

    def __init__(self) -> None:
        self.chromosomes = []

    def generate_population_from_data(self, population_size: int, subjects: Dict[str, Subject], professors: Dict[str, Professor], classrooms: Dict[str, Classroom], classgroups: Dict[str, ClassGroup]) -> None:
        for _ in range(population_size):
            chromosome = Chromosome()
            for classgroup_name, classgroup in classgroups.items(): # Para cada turma com x alunos
                for subject_name, subject in subjects.items(): # Para cada disciplina
                    for professor_name in subject.professors: # Para cada professor que habilitado para dar a disciplina
                        professor = professors[professor_id]
                        for classroom_name, classroom in classrooms.items(): # Para cada sala
                            for hous in range(subject.course_load // 36): # Para cada aula na semana (1 ou 2 aulas por semana a depender da carga horária da disciplina)
                                day = random.choice(list(professor.time_available.keys())) # Dia aleatório dos dias disponíveis do professor
                                time = random.choice(professor.time_available[day]) # Horário aleatório dos horários disponíveis do professor
                                gene = Gene(professor.name, subject.name, day, time, classgroup, classroom) 
                            chromosome.add_gene(gene)
            self.add_chromosome(chromosome)

    def add_chromosome(self, chromosome: Chromosome, fitness: float = 0) -> None:
        self.chromosomes.append((chromosome, fitness))

In [ ]:
@dataclass
class EvaluationMetrics:
    iterations: int = 0
    best_iteration: int = 0
    avg_conflicts_history: List[int] = []
    avg_elite_fitness_history: List[float] = []
    time_to_evaluate: List[float] = []
    best_timetable: Any = None
    best_conflicts: int = 0
    best_elite_fitness: float|None = None
    time_to_converge: float = 0

## 0.3 - Custom Classes

In [31]:
class MetricsEvaluator:
    def __init__(self, model: str) -> None:
        self.model: str = model
        self.metrics: EvaluationMetrics = EvaluationMetrics()
        self.__start_timer()

    def __repr__(self) -> str:
        return f"MetricsEvaluator({self.model})"
    
    def __str__(self) -> str:
        return f"MetricsEvaluator of Model: {self.model} with Metrics: {self.metrics}"
    
    def __start_timer(self) -> None:
        self.start_time = time.time()
    
    def __stop_timer(self):
        self.metrics.time_to_converge = (time.time() - self.start_time) / 60

    def start_iteration_timer(self) -> None:
        self.iteration_start_time = time.time()

    def stop_iteration_timer(self) -> None:
        self.metrics.time_to_evaluate.append((time.time() - self.iteration_start_time) / 60)

    def update_metrics(self, iteration: int, avg_conflicts: int, avg_elite_fitness: float|None = None) -> None:
        self.metrics.iterations = iteration
        self.metrics.avg_conflicts_history.append(avg_conflicts)
        if avg_elite_fitness is not None:
            self.metrics.avg_elite_fitness_history.append(avg_elite_fitness)

    def update_best_metric(self, best_iteration: int, best_timetable: Any, best_conflicts: int, best_elite_fitness: float|None = None) -> None:
        self.metrics.best_iteration = best_iteration
        self.metrics.best_timetable = best_timetable
        self.metrics.best_conflicts = best_conflicts
        if best_elite_fitness is not None:
            self.metrics.best_elite_fitness = best_elite_fitness
        self.__stop_timer()

    def save_metrics(self, file_path: str) -> None:
        with open(file_path, "w") as file:
            json.dump(self.metrics, file, indent=4)

## 0.4 - Custom Functions

In [32]:
def load_data(file_path, data_class) -> dict:
    with open(file_path, 'r') as file:
        data = json.load(file)
        for key, value in data.items():
            data[key] = data_class(key, value)
        return data

In [33]:
def calculate_quantity_of_classes(subjects: Dict[str, Subject], classrooms: Dict[str, Classroom]) -> float:
    quantity_of_classes: float = 0
    for subject in subjects.values():
        quantity_of_classes += subject.course_load // 1.5
    quantity_of_classes = quantity_of_classes * len(classrooms)
    print(f"Quantity of classes: {quantity_of_classes}")
    return quantity_of_classes

In [34]:
def generate_initial_population(population_size: int, subjects: Dict[str, Subject], professors: Dict[str, Professor], classrooms: Dict[str, Classroom], classgroups: Dict[str, ClassGroup]) -> Population:
    print(f"Creating population with {population_size} chromosomes, {len(subjects)} subjects, {len(professors)} professors and {len(classrooms)} classrooms")
    population = Population()
    for _ in range(population_size):
        chromosome = Chromosome()
        for subject, subject_data in subjects.items():
            for professor in subject_data.professors:
                for classgroup in classgroups.values():
                    for _ in range((subject_data.course_load // 36)):
                        for classroom, classroom_data in classrooms.items():
                            if classgroup.students <= classroom_data.capacity:
                                day, time = random.choice(list(set(professors[professor].availability) & set(classroom_data.availability)))
                                gene = Gene(professors[professor], subject_data, day, time, classgroup, classroom_data)
                                chromosome.add_gene(gene)
        population.add_chromosome(chromosome)
    #ic(population)
    return population

In [35]:
def calculate_chromosome_fitness(chromosome: Chromosome) -> int:
    conflicts: int = 0

    timeslot_count: dict = {}
    room_usage: dict = {}

    for gene in chromosome.genes:
        # Contar uso de horários por professor
        if (gene.professor.name, gene.day, gene.time) in timeslot_count:
            timeslot_count[(gene.professor.name, gene.day, gene.time)] += 1
        else:
            timeslot_count[(gene.professor.name, gene.day, gene.time)] = 1

        # Contar uso de salas
        if (gene.subject.name, gene.day, gene.time) in room_usage:
            room_usage[(gene.subject.name, gene.day, gene.time)] += 1
        else:
            room_usage[(gene.subject.name, gene.day, gene.time)] = 1

    for count in timeslot_count.values():
        if count > 1:
            conflicts += (count - 1)

    for count in room_usage.values():
        if count > 1:
            conflicts += (count - 1)
            
    for gene in chromosome.genes:
        if gene.classgroup.students > gene.classroom.capacity:
            conflicts += 1

    return conflicts
    

In [36]:
def calculate_chromosome_elite_fitness(chromosome: Chromosome, conflicts: int|None = None) -> float:
    fitness = 1.0
    if conflicts is None:
        conflicts = calculate_chromosome_fitness(chromosome)

    # Reduzir a fitness por conflito
    fitness -= conflicts * 0.1
    
    if fitness < 0:
        fitness = 0

    return fitness

In [37]:
def selection(population: Population) -> Population:
    mating_pool: Population = Population()
    
    population_fitness = [(chromosome, calculate_chromosome_elite_fitness(chromosome)) for chromosome, _ in population.chromosomes]
    population_fitness.sort(key=lambda x: x[1], reverse=True)

    for i in range(len(population_fitness)):
        if random.random() < (i / len(population_fitness)):
            mating_pool.add_chromosome(population_fitness[i][0])
    return mating_pool

In [38]:
def crossover(mating_pool: Population, CROSSOVER_RATE: float) -> Population:
    offspring: Population = Population()
    for _ in range(len(mating_pool) // 2):
        parent1 = random.choice(mating_pool)
        parent2 = random.choice(mating_pool)
        if random.random() < CROSSOVER_RATE:
            point = random.randint(1, len(parent1.genes) - 1)
            child1 = Chromosome()
            child2 = Chromosome()
            child1.genes = parent1.genes[:point] + parent2.genes[point:]
            child2.genes = parent2.genes[:point] + parent1.genes[point:]
            offspring.extend([child1, child2])
        else:
            offspring.extend([parent1, parent2])
    return offspring

In [39]:
def mutate(mating_pool: List[Chromosome], MUTATION_RATE: float, professors: Dict[str, Professor], subjects: Dict[str, Subject], classrooms: Dict[str, Classroom]) -> List[Chromosome]:
    for chromosome in mating_pool:
        for gene in chromosome.genes:
            if random.random() < MUTATION_RATE:
                # available_times = list(set(professors[gene.professor.name].time_available) & set(classrooms))
                available_times = list(set(professors[gene.professor.name].availability) & set(classrooms[gene.classroom.name].availability))
                day, time = random.choice(available_times)
                gene.day = day
                gene.time = time
    return mating_pool

In [40]:
def replace_population(population: Population, mating_pool: List[Chromosome]) -> List[Chromosome]:
    population_fitness = [(chromosome, calculate_chromosome_elite_fitness(chromosome)) for chromosome in population.chromosomes]
    population_fitness.sort(key=lambda x: x[1])

    mating_pool_fitness = [(chromosome, calculate_chromosome_elite_fitness(chromosome)) for chromosome in mating_pool]
    mating_pool_fitness.sort(key=lambda x: x[1], reverse=True)

    for i in range(len(mating_pool_fitness)):
        if mating_pool_fitness[i][1] > population_fitness[i][1]:
            population_fitness[i] = mating_pool_fitness[i]

    return [chromosome for chromosome, fitness in population_fitness]

# 1 Model

## 1.1 Load Data

In [41]:
relationship = {
    PROFESSORS_FILE_PATH: Professor,
    SUBJECTS_FILE_PATH: Subject,
    CLASSROOMS_FILE_PATH: Classroom,
    CLASSGROUPS_FILE_PATH: ClassGroup
}

In [42]:
professors = load_data(PROFESSORS_FILE_PATH, relationship[PROFESSORS_FILE_PATH])
professors

{'Neto': Professor(id=1, name='Neto', time_available={'Segunda': [19, 20, 21], 'Terça': [18, 21], 'Quarta': [18, 19, 20, 21], 'Sexta': [18, 19, 20, 21]}),
 'Chiquinho': Professor(id=2, name='Chiquinho', time_available={'Segunda': [18, 19, 20], 'Terça': [18, 19, 20], 'Quarta': [18, 19, 20], 'Quinta': [18, 19, 20], 'Sexta': [18, 19, 20]}),
 'Maria': Professor(id=3, name='Maria', time_available={'Segunda': [18, 19, 20], 'Terça': [18, 19, 20], 'Quarta': [18, 19, 20], 'Quinta': [18, 19, 20], 'Sexta': [18, 19, 20]}),
 'Joao': Professor(id=4, name='Joao', time_available={'Segunda': [18, 19, 20, 21], 'Terça': [18, 19, 20, 21], 'Quarta': [18, 19, 20, 21], 'Quinta': [18, 19, 20, 21], 'Sexta': [18, 19, 20, 21]})}

In [43]:
subjects = load_data(SUBJECTS_FILE_PATH, relationship[SUBJECTS_FILE_PATH])
subjects

{'Redes': Subject(id=1, name='Redes', professors=['Neto'], course_load=72),
 'Logica': Subject(id=2, name='Logica', professors=['Chiquinho'], course_load=36),
 'Matematica': Subject(id=3, name='Matematica', professors=['Maria'], course_load=36),
 'Fisica': Subject(id=4, name='Fisica', professors=['Joao'], course_load=72)}

In [44]:
classrooms = load_data(CLASSROOMS_FILE_PATH, relationship[CLASSROOMS_FILE_PATH])
classrooms

{'Sala 101': Classroom(id=1, name='Sala 101', time_available={'Segunda': [18, 19, 20, 21], 'Terça': [18, 19, 20, 21], 'Quarta': [18, 19, 20, 21], 'Quinta': [18, 19, 20, 21], 'Sexta': [18, 19, 20, 21]}, capacity=60),
 'Sala 102': Classroom(id=2, name='Sala 102', time_available={'Segunda': [18, 19, 20, 21], 'Terça': [18, 19, 20], 'Quarta': [19, 20, 21], 'Quinta': [18, 19, 20, 21], 'Sexta': [19, 20, 21]}, capacity=30),
 'Sala 103': Classroom(id=3, name='Sala 103', time_available={'Segunda': [18, 19, 20, 21], 'Terça': [18, 19, 20, 21], 'Quarta': [18, 19, 20, 21], 'Quinta': [18, 19, 20, 21], 'Sexta': [18, 19, 20, 21]}, capacity=40)}

In [45]:
classgroups = load_data(CLASSGROUPS_FILE_PATH, relationship[CLASSGROUPS_FILE_PATH])
classgroups

{'Alpha': ClassGroup(id=1, name='Alpha', students=20),
 'Zeta': ClassGroup(id=2, name='Zeta', students=18),
 'Shaw': ClassGroup(id=3, name='Shaw', students=22),
 'Kira': ClassGroup(id=4, name='Kira', students=25),
 'Vex': ClassGroup(id=5, name='Vex', students=19)}

## 1.2 Model Definition

In [46]:
POPULATION_SIZE = 100
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8
GENERATIONS = 1000
FITNESS_THRESHOLD = 0.9

In [47]:
NUM_CLASSES = calculate_quantity_of_classes(subjects, classrooms)

Quantity of classes: 432.0


In [50]:
def genetic_algorithm(population_size: int, mutation_rate: float, crossover_rate: float, generations: int, fitness_threshold: float, subjects: Dict[str, Subject], professors: Dict[str, Professor], classrooms: Dict[str, Classroom], classgroups: Dict[str, ClassGroup]) -> Tuple[Chromosome, MetricsEvaluator]:

    metrics_evaluator = MetricsEvaluator("Genetic Algorithm")
    metrics_evaluator.start_timer()
    population = generate_initial_population(population_size, subjects, professors, classrooms, classgroups)
    best_chromosome = None
    first_gereration = True
    last_good_elite_fitness = 0
    last_good_chromosome: Chromosome|None = None
    last_good_generation = 0
    last_good_conflicts = 0
    for generation in range(0, generations):
        print(f"Evaluating generation: {generation}")
        metrics_evaluator.start_iteration_timer()
        if not first_gereration:
            mating_pool = selection(population)
            mating_pool = crossover(mating_pool, crossover_rate)
            mating_pool = mutate(mating_pool, mutation_rate, professors, subjects, classrooms)
            population = replace_population(population, mating_pool)
    
        # Calcula a "nota" (fitness) de cada conjunto de horários
        population_conflicts = []
        population_elite_fitness = []
        #print(population)
        for chromosome in population.chromosomes:
            conflicts = calculate_chromosome_fitness(chromosome)
            population_conflicts.append(conflicts)
            elite_fitness = calculate_chromosome_elite_fitness(chromosome, conflicts)
            population_elite_fitness.append(elite_fitness)
        print(f"Population conflicts: {population_conflicts}")
        print(f"Population elite fitness: {population_elite_fitness}")

        # Encontra a melhor "nota" atual
        conflicts_elite_fitnesss = min(population_conflicts)
        max_elite_fitness = max(population_elite_fitness)
        print(f"Generation: {generation}, Conflicts: {conflicts_elite_fitnesss}, Elite Fitness: {max_elite_fitness}")

        if max_elite_fitness > last_good_elite_fitness:
            print(f"New best elite fitness found in generation {generation}")
            last_good_elite_fitness = max_elite_fitness
            last_good_chromosome = population.chromosomes[population_elite_fitness.index(max_elite_fitness)]
            last_good_generation = generation
            last_good_conflicts = conflicts_elite_fitnesss

            if max_elite_fitness >= fitness_threshold:
                print(f"Fitness threshold reached in generation {generation}")
                best_elite_fitness = max_elite_fitness
                best_chromosome = last_good_chromosome
                best_generation = last_good_generation
                best_conflicts = last_good_conflicts
        
        avg_elite_fitness = sum(population_elite_fitness) / len(population_elite_fitness)
        avg_conflicts = sum(population_conflicts) / len(population_conflicts)
        metrics_evaluator.update_metrics(generation, avg_conflicts, avg_elite_fitness)
        metrics_evaluator.stop_iteration_timer()
        if first_gereration:
            first_gereration = False

    if best_chromosome is None:
        best_elite_fitness = last_good_elite_fitness
        best_chromosome = last_good_chromosome
        best_generation = last_good_generation
        best_conflicts = last_good_conflicts

    metrics_evaluator.update_best_metric(best_generation, best_chromosome, best_conflicts, best_elite_fitness)
    
    metrics_evaluator.stop_timer()
    print(f"Best Generation: {best_generation}, Best chromosome: {best_chromosome}, Conflicts: {best_conflicts}, Elite Fitness: {best_elite_fitness}")
    return best_chromosome, metrics_evaluator

## 2 Experiment

In [51]:
best_timetable, metrics = genetic_algorithm(
    POPULATION_SIZE,
    MUTATION_RATE,
    CROSSOVER_RATE,
    GENERATIONS,
    FITNESS_THRESHOLD,
    subjects,
    professors,
    classrooms,
    classgroups
)

Creating population with 100 chromosomes, 4 subjects, 4 professors and 3 classrooms
Evaluating generation: 0
Population conflicts: [90, 84, 88, 86, 88, 80, 88, 82, 78, 80, 82, 84, 86, 94, 82, 88, 86, 92, 86, 94, 84, 80, 82, 88, 80, 94, 86, 84, 88, 90, 82, 84, 88, 92, 76, 90, 82, 88, 86, 92, 96, 92, 74, 78, 88, 82, 90, 90, 96, 78, 80, 84, 86, 92, 76, 84, 86, 90, 84, 86, 88, 96, 86, 88, 74, 90, 80, 86, 86, 90, 86, 90, 84, 84, 90, 88, 94, 86, 86, 92, 80, 90, 90, 88, 82, 84, 88, 92, 92, 84, 94, 84, 92, 86, 86, 80, 92, 86, 86, 88]
Population elite fitness: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Generation: 0, Conflicts: 74, Elite Fitness: 0
Evaluating generation: 1


AttributeError: 'list' object has no attribute 'chromosomes'

In [27]:
print(metrics)

MetricsEvaluator of Model: Genetic Algorithm with Metrics: {'time_to_converge': 0.05346132119496663, 'best_conflicts': 0, 'best_elite_fitness': 0, 'best_timetable': None, 'iterations': 999, 'best_iteration': 0, 'avg_conflicts_history': [87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 87.22, 

# 3 Final Model

In [28]:

class MetricsEvaluator:
    def __init__(self, model: str) -> None:
        self.model = model
        self.metrics = {
            "time_to_converge": 0,
            "best_fitness": 0,
            "best_chromosome": None
        }

    def __repr__(self) -> str:
        return f"MetricsEvaluator({self.model})"
    
    def __str__(self) -> str:
        return f"MetricsEvaluator of Model: {self.model} with Metrics: {self.metrics}"

    def start_timer(self):
        self.start_time = time.time()

    def stop_timer(self):
        self.metrics["time_to_converge"] = time.time() - self.start_time // 60

    def update_metrics(self, best_chromosome: Chromosome, best_fitness: float):
        self.metrics["best_fitness"] = best_fitness
        self.metrics["best_chromosome"] = best_chromosome
        
class GeneticAlgorithm:
    def __init__(self, population_size: int, mutation_rate: float, crossover_rate: float, generations: int, fitness_threshold: float, subjects: Dict[str, Subject], professors: Dict[str, Professor], classrooms: Dict[str, Classroom], classgroups: Dict[str, ClassGroup]) -> None:
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.generations = generations
        self.fitness_threshold = fitness_threshold
        self.subjects = subjects
        self.professors = professors
        self.classrooms = classrooms
        self.classrooms = classrooms
        self.classgroups = classgroups


    def generate_initial_population(self) -> Population:
        print(f"Creating population with {self.population_size} chromosomes, {len(self.subjects)} subjects, {len(self.professors)} professors and {len(self.classrooms)} classrooms")
        population = Population()
        for _ in range(self.population_size):
            chromosome = Chromosome()
            for subject, subject_data in self.subjects.items():
                for professor in subject_data.professors:
                    for classgroup in self.classgroups.values():
                        for _ in range((subject_data.course_load // 36)):
                            for classroom, classroom_data in self.classrooms.items():
                                if classgroup.students <= classroom_data.capacity:
                                    day, time = random.choice(list(set(self.professors[professor].availability) & set(classroom_data.availability)))
                                    gene = Gene(self.professors[professor], subject_data, day, time)
                                    chromosome.add_gene(gene)
            population.add_chromosome(chromosome)
        return population
    
    def calculate_chromosome_fitness(self, chromosome: Chromosome) -> float:
        fitness = 1.0
        conflicts = 0
        timeslot_count = {}
        room_usage = {}
        for gene in chromosome.genes:
            # Contar uso de horários por professor
            if (gene.professor.name, gene.day, gene.time) in timeslot_count:
                timeslot_count[(gene.professor.name, gene.day, gene.time)] += 1
            else:
                timeslot_count[(gene.professor.name, gene.day, gene.time)] = 1
            # Contar uso de salas
            if (gene.subject.name, gene.day, gene.time) in room_usage:
                room_usage[(gene.subject.name, gene.day, gene.time)] += 1
            else:
                room_usage[(gene.subject.name, gene.day, gene.time)] = 1
        for count in timeslot_count.values():
            if count > 1:
                conflicts += (count - 1)
        for count in room_usage.values():
            if count > 1:
                conflicts += (count - 1)
        # Verificar capacidade da sala
        for gene in chromosome.genes:
            if gene.classgroup.students > gene.classroom.capacity:
                conflicts += 1
        # Reduzir a fitness por conflito
        fitness -= conflicts * 0.1
        if fitness < 0:
            fitness = 0
        return fitness
    
    def selection(self, population: Population) -> List[Chromosome]:
        mating_pool = []
        population_fitness = [(chromosome, self.calculate_chromosome_fitness(chromosome)) for chromosome in population.chromosomes]
        population_fitness.sort(key=lambda x: x[1], reverse=True)
        for i in range(len(population_fitness)):
            if random.random() < (i / len(population_fitness)):
                mating_pool.append(population_fitness[i][0])
        return mating_pool
    
    def crossover(self, mating_pool: List[Chromosome]) -> List[Chromosome]:
        offspring = []
        for _ in range(len(mating_pool) // 2):
            parent1 = random.choice(mating_pool)
            parent2 = random.choice(mating_pool)
            if random.random() < self.crossover_rate:
                point = random.randint(1, len(parent1.genes) - 1)
                child1 = Chromosome()
                child2 = Chromosome()
                child1.genes = parent1.genes[:point] + parent2.genes[point:]
                child2.genes = parent2.genes[:point] + parent1.genes[point:]
                offspring.extend([child1, child2])
            else:
                offspring.extend([parent1, parent2])
        return offspring
    
    def mutate(self, mating_pool: List[Chromosome]) -> List[Chromosome]:
        for chromosome in mating_pool:
            for gene in chromosome.genes:
                if random.random() < self.mutation_rate:
                    # available_times = list(set(self.professors[gene.professor.name].time_available) & set(self.classrooms))
                    available_times = list(set(self.professors[gene.professor.name].availability) & set(self.classrooms[gene.classroom.name].availability))
                    day, time = random.choice(available_times)
                    gene.day = day
                    gene.time = time
        return mating_pool
    
    def replace_population(self, population: Population, mating_pool: List[Chromosome]) -> List[Chromosome]:
        population_fitness = [(chromosome, self.calculate_chromosome_fitness(chromosome)) for chromosome in population.chromosomes]
        population_fitness.sort(key=lambda x: x[1])
        mating_pool_fitness = [(chromosome, self.calculate_chromosome_fitness(chromosome)) for chromosome in mating_pool]
        mating_pool_fitness.sort(key=lambda x: x[1], reverse=True)
        for i in range(len(mating_pool_fitness)):
            if mating_pool_fitness[i][1] > population_fitness[i][1]:
                population_fitness[i] = mating_pool_fitness[i]
        return [chromosome for chromosome, fitness in population_fitness]

    def run(self) -> Tuple[Chromosome, MetricsEvaluator]:
        metrics_evaluator = MetricsEvaluator("Genetic Algorithm")
        metrics_evaluator.start_timer()
        population = generate_initial_population(self.population_size, self.subjects, self.professors, self.classrooms)
        best_chromosome = None
        for generation in range(0, self.generations):
            print(f"Evaluating generation: {generation}")

            # Calcula a "nota" (fitness) de cada conjunto de horários
            population_fitness = [calculate_chromosome_fitness(chromosome) for chromosome in population.chromosomes]

            # Encontra a melhor "nota" atual
            best_fitness = max(population_fitness)

            if best_fitness >= self.fitness_threshold:
                print(f"Fitness threshold reached in generation {generation}")
                best_chromosome = population.chromosomes[population_fitness.index(best_fitness)]
                break

            mating_pool = selection(population)
            mating_pool = crossover(mating_pool, self.crossover_rate)
            mating_pool = mutate(mating_pool, self.mutation_rate, self.professors, self.subjects, self.classrooms)
            population = replace_population(population, mating_pool)

        if best_chromosome is None:
            best_chromosome = population.chromosomes[population_fitness.index(max(population_fitness))]
        metrics_evaluator.stop_timer()
        metrics_evaluator.update_metrics(best_chromosome, best_fitness)
        print(f"Best chromosome: {best_chromosome}, Fitness: {best_fitness}")
        return best_chromosome, metrics_evaluator